In [1]:
# A script to try to find all transit agencies that have service in the GTFS feed 
# that differs from what is stated on their Holiday Service website.

In [3]:
import json
import siuba
from calitp_data_analysis.tables import tbls
from siuba import *

In [58]:
# Get data from airtable
orgs_df = (
    tbls.mart_transit_database.dim_organizations()
        >> filter(_._is_current == True)
        >> filter(_.public_currently_operating_fixed_route == True)
        >> collect()
)

services_df = (
    tbls.mart_transit_database.dim_services()
        >> filter(_._is_current == True)
        >> filter(_.public_currently_operating_fixed_route == True)
        >> collect()
)

In [23]:
# Get list of holiday service from Airtable extract since it won't be in the mart
with open('./services.json') as f:
    services_json = json.loads(f.read())

In [53]:
# refactor services_json so it can be looked up by key
services_by_name = {}
for service in services_json:
    services_by_name[service['name'].strip()] = service

In [44]:
# Define holidays
holidays = [
    {
        'name': "Veteran's Day (Observed)",
        'date': '2023-11-10',
        'airtable_column': 'holiday_schedule___veterans_day__observed_'
    }, {
        'name': "Veteran's Day",
        'date': '2023-11-11',
        'airtable_column': 'holiday_schedule___veterans_day'
    }, {
        'name': "Thanksgiving Day",
        'date': '2023-11-23',
        'airtable_column': 'holiday_schedule___thanksgiving_day'
    }, {
        'name': "Day After Thanksgiving",
        'date': '2023-11-24',
        'airtable_column': 'holiday_schedule___day_after_thanksgiving_day'
    }, {
        'name': "Christmas Eve",
        'date': '2023-12-24',
        'airtable_column': 'holiday_schedule___christmas_eve'
    }, {
        'name': "Christmas Day",
        'date': '2023-12-25',
        'airtable_column': 'holiday_schedule___christmas_day'
    }, {
        'name': "New Year's Eve",
        'date': '2023-12-31',
        'airtable_column': 'holiday_schedule___new_year_s_eve'
    }, {
        'name': "New Year's Day",
        'date': '2024-01-01',
        'airtable_column': 'holiday_schedule___new_year_s_day'
    }, {
        'name': "MLK Day",
        'date': '2024-01-15',
        'airtable_column': 'holiday_schedule___mlk_day'
    }        
]

In [70]:
services_holiday_analysis = []

# for each active service
for i, r in services_df.iterrows():
    # TODO: skip services that are seasonal outside of analysis period
    
    # initialize output variable
    result = {
        'Service': r['name']
    }
    
    # match to raw services
    services_with_holiday_info = services_by_name[r['name']]
    
    # TODO: get GTFS Schedule for service (number of trips per day)
    
    # for each holiday type
    for holiday in holidays:
        # obtain reference non-holiday number of trips per day from GTFS
        num_trips_non_holiday = 0
        
        # obtain number of trips on holiday day from GTFS
        num_trips_on_holiday = 0
        
        # do gtfs comparison
        gtfs_holiday_schedule = 'Regular service'
        if num_trips_on_holiday == 0:
            gtfs_holiday_schedule = 'No service'
        elif num_trips_on_holiday < num_trips_non_holiday:
            gtfs_holiday_schedule = 'Reduced service'
        
        # add website data
        result[holiday['name']] = {
            'gtfs_service': gtfs_holiday_schedule,
            'website_service': services_with_holiday_info[holiday['airtable_column']]
        }
        
    services_holiday_analysis.append(result)

In [69]:
services_holiday_analysis[8]

{'Service': 'GTrans',
 "Veteran's Day (Observed)": {'gtfs_service': 'No service',
  'website_service': 'Regular service'},
 "Veteran's Day": {'gtfs_service': 'No service',
  'website_service': 'Regular service'},
 'Thanksgiving Day': {'gtfs_service': 'No service',
  'website_service': 'Reduced service'},
 'Day After Thanksgiving': {'gtfs_service': 'No service',
  'website_service': 'Regular service'},
 'Christmas Eve': {'gtfs_service': 'No service',
  'website_service': 'Regular service'},
 'Christmas Day': {'gtfs_service': 'No service',
  'website_service': 'Reduced service'},
 "New Year's Eve": {'gtfs_service': 'No service',
  'website_service': 'Regular service'},
 "New Year's Day": {'gtfs_service': 'No service',
  'website_service': 'Reduced service'},
 'MLK Day': {'gtfs_service': 'No service',
  'website_service': 'Regular service'}}